In [13]:
# imports
import pandas as pd
import numpy as np
import os, time,json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
import glob

In [2]:
# custom functions
def write_json(new_data, filename): 
    "Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/" 
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

def get_movie_with_rating(movie_id):

    #get movie object for the current id
    movie = tmdb.Movies(movie_id)

    #save the .info and .releases dictionaries
    info = movie.info()
    releases = movie.releases()

    #loop through countries in releases
    for country in releases["countries"]:
        #country == US
        if country["iso_3166_1"] == "US":
            #save certification
            info["certification"] = country["certification"]
    return info

In [3]:
# getting login credentials
with open("/Users/chery/.secret/tmdb_api.json", "r") as file:
    login = json.load(file)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [4]:
# setting API key
tmdb.API_KEY =  login['api-key']

In [5]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'final_tmdb_data_2010.csv.gz',
 'final_tmdb_data_2011.csv.gz',
 'final_tmdb_data_2012.csv.gz',
 'final_tmdb_data_2013.csv.gz',
 'final_tmdb_data_2014.csv.gz',
 'final_tmdb_data_2015.csv.gz',
 'final_tmdb_data_2016.csv.gz',
 'final_tmdb_data_2017.csv.gz',
 'final_tmdb_data_2018.csv.gz',
 'final_tmdb_data_2019.csv.gz',
 'title_akas_combined.csv.gz',
 'title_basics_combined.csv.gz',
 'title_ratings_combined.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2010.json',
 'tmdb_api_results_2011.json',
 'tmdb_api_results_2012.json',
 'tmdb_api_results_2013.json',
 'tmdb_api_results_2014.json',
 'tmdb_api_results_2015.json',
 'tmdb_api_results_2016.json',
 'tmdb_api_results_2017.json',
 'tmdb_api_results_2018.json',
 'tmdb_api_results_2019.json',
 'tmdb_results_combined.csv.gz']

In [6]:
# Reading in the basics df
dfbasics = pd.read_csv("Data/title_basics_combined.csv.gz")
dfbasics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
4,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008.0,NaN,94,Horror


In [11]:
# creating list of years to get and an errors list
yearsneeded = [2010,2011,2012,2013,2014,2015,2016,2017,2018,]
errors = []

In [ ]:
# start of outer loop
for YEAR in tqdm_notebook(yearsneeded, desc='YEARS', position=0):
    
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    
    # If it does not exist: create it
    if file_exists == False:
        
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
            
    #Saving new year as the current df
    df = dfbasics.loc[dfbasics['startYear']==YEAR].copy()

    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    #start of inner loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieving the data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Appending the results to existing file using the predefined function in notebook
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

In [ ]:
print(f"- Total errors encountered: {len(errors)}")

# Combining all files

In [15]:
# gettng list of files to combine
q = "Data/final_tmdb_data_201*.csv.gz"
chunked_files = glob.glob(q)
chunked_files[:10]

['Data\\final_tmdb_data_2010.csv.gz',
 'Data\\final_tmdb_data_2011.csv.gz',
 'Data\\final_tmdb_data_2012.csv.gz',
 'Data\\final_tmdb_data_2013.csv.gz',
 'Data\\final_tmdb_data_2014.csv.gz',
 'Data\\final_tmdb_data_2015.csv.gz',
 'Data\\final_tmdb_data_2016.csv.gz',
 'Data\\final_tmdb_data_2017.csv.gz',
 'Data\\final_tmdb_data_2018.csv.gz',
 'Data\\final_tmdb_data_2019.csv.gz']

In [16]:
df_combinedapires = pd.concat([pd.read_csv(file, index_col=0) for file in chunked_files])
df_combinedapires

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
imdb_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0146592,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,163942.0,hu,Pál Adrienn,"Piroska is an overweight, alienated nurse who ...",...,0.0,136.0,"[{'english_name': 'Hungarian', 'iso_639_1': 'h...",Released,NaN,Adrienn Pál,0.0,5.7,7.0,NaN
tt0154039,0.0,/anEsO8XT4wK7G9ScZjfvDeCztNg.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",NaN,260823.0,hu,Oda az igazság,Concerning the Mátyás era in Hungarian history...,...,0.0,100.0,"[{'english_name': 'Hungarian', 'iso_639_1': 'h...",Released,NaN,So Much for Justice!,0.0,4.0,1.0,NaN
tt0162942,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,61200.0,en,A zöld sárkány gyermekei,"Mate, a lonely, divorced real estate agent is ...",...,0.0,0.0,[],Released,NaN,Children of the Green Dragon,0.0,5.7,3.0,NaN
tt0312305,0.0,/lqUbt2cy2pnqvxKefbQAtxLS0WA.jpg,NaN,0.0,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",http://www.qqthemovie.com/,23738.0,en,Quantum Quest: A Cassini Space Odyssey,"Dave, a young photon, who is forced out of the...",...,0.0,45.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Quantum Quest: A Cassini Space Odyssey,0.0,7.9,8.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9913872,0.0,/a0kySBmMZ77AqmNA3pDu45GsbCp.jpg,NaN,0.0,"[{'id': 27, 'name': 'Horror'}]",NaN,800030.0,es,De la piel del Diablo,NaN,...,0.0,0.0,[],Released,NaN,De la piel del Diablo,0.0,8.2,3.0,NaN
tt9914942,0.0,/tjHUeULyw2vtS1DXFp0OHfhUzRX.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,598553.0,ca,La vida sense la Sara Amat,"Pep, a 13-year-old boy, is in love with a girl...",...,0.0,71.0,"[{'english_name': 'Catalan', 'iso_639_1': 'ca'...",Released,NaN,Life without Sara Amat,0.0,7.7,8.0,NaN
tt9915790,0.0,/9Zqup9L9zGMyDijAeKFkelcknp.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,591918.0,bn,Bobbyr Bondhura,Little Bobby goes missing after the separation...,...,0.0,106.0,[],Released,NaN,Bobbyr Bondhura,0.0,0.0,0.0,NaN


In [19]:
# Saving the combined dataframe as a CSV file
final_fname ='Data/tmdb_results_2010_2019_combined.csv.gz'
df_combinedapires.to_csv(final_fname, compression='gzip', index=True)